# Binary classification of commutative diagrams
## 1. Data pipeline

In [13]:
import pandas as pd
import numpy as np
#import torch
import os
import shutil
import random
import math
import tensorflow as tf
keras = tf.keras
#import tensorflow.keras as keras
import tensorflow_addons as tfa # Needed for Yogi optimizer
import matplotlib.pyplot as plt
print(tf.config.list_physical_devices('GPU'))

[]


### 1.1 Expand working directory with folders for training images

In [2]:
##
unsortedSamplesDirName = 'png-2021-all_contexts'
##

cwd = os.getcwd()

unsortedSamplesDir = os.path.join(cwd, unsortedSamplesDirName)
assert('positive' in os.listdir(unsortedSamplesDir) and 'negative' in os.listdir(unsortedSamplesDir))
unsortedPositiveSamplesDir = os.path.join(unsortedSamplesDir, 'positive')
unsortedNegativeSamplesDir = os.path.join(unsortedSamplesDir, 'negative')

sortedSamplesDir = os.path.join(cwd, 'sortedSamples')
os.makedirs(sortedSamplesDir, exist_ok=True)

trainDir = os.path.join(sortedSamplesDir, 'train')
testDir = os.path.join(sortedSamplesDir, 'test')
valDir = os.path.join(sortedSamplesDir, 'val')
os.makedirs(trainDir, exist_ok=True)
os.makedirs(testDir, exist_ok=True)
os.makedirs(valDir, exist_ok=True)

trainPositiveDir = os.path.join(trainDir, 'positive')
trainNegativeDir = os.path.join(trainDir, 'negative')
testPositiveDir = os.path.join(testDir, 'positive')
testNegativeDir = os.path.join(testDir, 'negative')
valPositiveDir = os.path.join(valDir, 'positive')
valNegativeDir = os.path.join(valDir, 'negative')
os.makedirs(trainPositiveDir, exist_ok=True)
os.makedirs(trainNegativeDir, exist_ok=True)
os.makedirs(testPositiveDir, exist_ok=True)
os.makedirs(testNegativeDir, exist_ok=True)
os.makedirs(valPositiveDir, exist_ok=True)
os.makedirs(valNegativeDir, exist_ok=True)

### 1.2 Copy images into train, test and validation folders
The code currently randomly truncates the greater partition between positive and negative to achieve parity.

In [3]:
##
trainRatio = 0.6
testRatio = 0.2
valRatio = 0.2
assert(trainRatio + testRatio + valRatio == 1.)

positiveRatio = 0.5 # Desired ratio of positive samples in the sorted data
##


positiveSamplesFilenameList = os.listdir(unsortedPositiveSamplesDir)
negativeSamplesFilenameList = os.listdir(unsortedNegativeSamplesDir)
random.shuffle(positiveSamplesFilenameList)
random.shuffle(negativeSamplesFilenameList)
numPositiveSamples = len(positiveSamplesFilenameList)
numNegativeSamples = len(negativeSamplesFilenameList)

if numPositiveSamples > numNegativeSamples:
    positiveSamplesFilenameList = positiveSamplesFilenameList[:numNegativeSamples]
    numPositiveSamples = len(positiveSamplesFilenameList)
elif numNegativeSamples > numPositiveSamples:
    negativeSamplesFilenameList = negativeSamplesFilenameList[:numPositiveSamples]
    numNegativeSamples = len(negativeSamplesFilenameList)
assert(numPositiveSamples == numNegativeSamples)
    
numSamples = numPositiveSamples + numNegativeSamples

numTrainSamples = math.floor(numSamples*trainRatio)
numTestSamples = math.floor(numSamples*testRatio)
numValSamples = math.floor(numSamples*valRatio)


unsortedSamplesInfo = {'posDir':unsortedPositiveSamplesDir, 'negDir':unsortedNegativeSamplesDir, 'posFilenameList':positiveSamplesFilenameList,
                       'negFilenameList':negativeSamplesFilenameList}

trainCopyInfo = {'num':numTrainSamples, 'posDir':trainPositiveDir, 'negDir':trainNegativeDir}
testCopyInfo = {'num':numTestSamples, 'posDir':testPositiveDir, 'negDir':testNegativeDir}
valCopyInfo = {'num':numValSamples, 'posDir':valPositiveDir, 'negDir':valNegativeDir}

def copyImagesInPartition(unsortedSamplesInfo:dict, partitionCopyInfo:dict):
    for _ in range(math.floor(partitionCopyInfo['num']*positiveRatio)):
        copyImage(unsortedSamplesInfo['posFilenameList'], unsortedSamplesInfo['posDir'], partitionCopyInfo['posDir'])
    for _ in range(math.floor(partitionCopyInfo['num']*(1. - positiveRatio))):
        copyImage(unsortedSamplesInfo['negFilenameList'], unsortedSamplesInfo['negDir'], partitionCopyInfo['negDir'])

def copyImage(sampleFilenameList, srcDir, dstDir):
    filename = sampleFilenameList.pop()
    src = os.path.join(srcDir, filename)
    dst = os.path.join(dstDir, filename)
    try:
        shutil.copyfile(src, dst)
    except PermissionError: # Ignores straggler files such as notebook checkpoints
        pass

sampleDirectorySizes = [len(directory) for directory in [
    os.listdir(trainPositiveDir), os.listdir(trainNegativeDir), os.listdir(testPositiveDir), os.listdir(testNegativeDir),
    os.listdir(testPositiveDir), os.listdir(testNegativeDir)]]

if all(size == 0 for size in sampleDirectorySizes):
    copyImagesInPartition(unsortedSamplesInfo, trainCopyInfo)
    copyImagesInPartition(unsortedSamplesInfo, testCopyInfo)
    copyImagesInPartition(unsortedSamplesInfo, valCopyInfo)
else:
    raise Exception("Sorted image directories are not empty.")

# TRASH: Is this actually more readable than the functions above?
"""
if all(size == 0 for size in sampleDirectorySizes):
    for _ in range(math.floor(numTrainSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(trainPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTrainSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(trainNegativeDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTestSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(testPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numTestSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(testNegativeDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numValSamples*positiveRatio)):
        filename = positiveSamplesFilenameList.pop()
        src = os.path.join(unsortedPositiveSamplesDir, filename)
        dst = os.path.join(valPositiveDir, filename)
        shutil.copyfile(src, dst)

    for _ in range(math.floor(numValSamples*(1. - positiveRatio))):
        filename = negativeSamplesFilenameList.pop()
        src = os.path.join(unsortedNegativeSamplesDir, filename)
        dst = os.path.join(valNegativeDir, filename)
        shutil.copyfile(src, dst)
else:
    raise Exception("Sorted image directories are not empty.")"""

Exception: Sorted image directories are not empty.

[*Optional*]: Test whether a sample handful of the images were copied to the correct folders

In [4]:
##
runImageCopyTest = False

comparisonSampleSize = 50
imageFiletype = 'png'
##

if runImageCopyTest:
    # These must be redefined here since they were popped
    positiveSamplesFilenameList = os.listdir(unsortedPositiveSamplesDir)
    negativeSamplesFilenameList = os.listdir(unsortedNegativeSamplesDir)

    sortedTrainPositiveFilenameList = os.listdir(trainPositiveDir)
    sortedTrainNegativeFilenameList = os.listdir(trainNegativeDir)
    sortedTestPositiveFilenameList = os.listdir(testPositiveDir)
    sortedTestNegativeFilenameList = os.listdir(testNegativeDir)
    sortedValPositiveFilenameList = os.listdir(valPositiveDir)
    sortedValNegativeFilenameList = os.listdir(valNegativeDir)
    random.shuffle(sortedTrainPositiveFilenameList)
    random.shuffle(sortedTrainNegativeFilenameList)
    random.shuffle(sortedTestPositiveFilenameList)
    random.shuffle(sortedTestNegativeFilenameList)
    random.shuffle(sortedValPositiveFilenameList)
    random.shuffle(sortedValNegativeFilenameList)

    allPositiveFilenameLists = [sortedTrainPositiveFilenameList, sortedTestPositiveFilenameList, sortedValPositiveFilenameList]
    allNegativeFilenameLists = [sortedTrainNegativeFilenameList, sortedTestNegativeFilenameList, sortedValNegativeFilenameList]

    def matchesImageFiletype(sampleFilename : str, imageFiletype : str): # Needed to ignore straggler files such as notebook checkpoints
        return sampleFilename[:len(imageFiletype)] == imageFiletype

    for fList in allPositiveFilenameLists:
        assert(all(sampleFilename in positiveSamplesFilenameList
                   for sampleFilename in fList[:comparisonSampleSize]
                   if matchesImageFiletype(sampleFilename, imageFiletype)))
    for fList in allNegativeFilenameLists:
        assert(all(sampleFilename in negativeSamplesFilenameList
                   for sampleFilename in fList[:comparisonSampleSize]
                   if matchesImageFiletype(sampleFilename, imageFiletype)))
    print('Test was succesful!')

### 1.3 Data generators

The data generators themselves can rescale the input pixel values to the [0, 1] range. Note that the pretrained baseline models expect certain kinds of input - Respectively, EfficientNet excepts pixel floats in the [0-255] range to be passed through the keras.applications.resnet_v2.preprocess_input() function, and ResNet expects its input to be raw, which will have its pixel values rescaled to the (-1, 1) range. Thus, a seperate set of data generators are made for them which do net rescale their input.

In [5]:
##
batchSize = 75
##

trainDataGeneratorFactory = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
testDataGeneratorFactory = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valDataGenerator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

trainDataGenerator = trainDataGeneratorFactory.flow_from_directory( # Batch generator
    trainDir,
    target_size = (150, 150), # TODO: What size do I want?
    batch_size = batchSize,
    class_mode = 'binary')
testDataGenerator = testDataGeneratorFactory.flow_from_directory(
    testDir,
    target_size = (150, 150),
    batch_size = batchSize,
    class_mode = 'binary')
valDataGenerator = testDataGeneratorFactory.flow_from_directory(
    valDir,
    target_size = (150, 150),
    batch_size = batchSize,
    class_mode = 'binary')

baselineTrainDataGeneratorFactory = keras.preprocessing.image.ImageDataGenerator()
baselineTestDataGeneratorFactory = keras.preprocessing.image.ImageDataGenerator()
baselineValDataGeneratorFactory = keras.preprocessing.image.ImageDataGenerator()

baselineTrainDataGenerator = baselineTrainDataGeneratorFactory.flow_from_directory( # Batch generator
    trainDir,
    target_size = (150, 150), # TODO: What size do I want?
    batch_size = batchSize,
    class_mode = 'binary')
baselineTestDataGenerator = baselineTestDataGeneratorFactory.flow_from_directory(
    testDir,
    target_size = (150, 150),
    batch_size = batchSize,
    class_mode = 'binary')
baselineValDataGenerator = baselineValDataGeneratorFactory.flow_from_directory(
    valDir,
    target_size = (150, 150),
    batch_size = batchSize,
    class_mode = 'binary')

Found 2121 images belonging to 2 classes.
Found 706 images belonging to 2 classes.
Found 724 images belonging to 2 classes.
Found 2121 images belonging to 2 classes.
Found 706 images belonging to 2 classes.
Found 724 images belonging to 2 classes.


## 2 Baseline model implementation
Two pretrained models, EfficientNet and ResNet, will be implemented and tested as baselines. They will be implemented both as feature extractors (i.e. outputting their feature maps for a seperate model) and with retrained top-layer classifiers.

### 2.1 Implementation of untuned but pretrained EfficientNet and ResNet

In [6]:
# OTHER IDEA: Try a baseline model trained on MNIST

#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#print(f'Using {device}.')

#torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b0', pretrained=True')

efficientNetB0Base = keras.applications.efficientnet.EfficientNetB0(
    include_top = False,
    input_shape = (150, 150, 3), # Input to pretrained model must have 3 channels
    weights = 'imagenet')
    #pooling = 'avg')
resNet50V2Base = keras.applications.resnet_v2.ResNet50V2(
    include_top = False,
    input_shape = (150, 150, 3), # Input to pretrained model must have 3 channels
    weights = 'imagenet')
    #pooling = 'avg')

efficientNetB0Base.trainable = False
resNet50V2Base.trainable = False

#efficientNetB0 = models.Sequential(name = 'efficientNetB0')
#efficientNetB0.add(tf.keras.layers.Conv2D(16, 7, strides = 3, padding = 'same'/'valid', activation="relu", trainable=True))

# TODO: The default final output is of shape 5x5x1280. I need to figure out how to downscale that into a single softmax output.

# keras.applications.resnet_v2.preprocess_input()
# When exactly should this be implemented?

### 2.2 Feature extraction function

In [7]:
def extractFeatures(model, generator, modelOutputShape:tuple, numSamples=None):
    if not numSamples:
        numSamples = generator.n
    if numSamples % batchSize != 0:
        numSamples -= (numSamples % batchSize)
    features = np.zeros(shape=(numSamples, *modelOutputShape)) # 5 5 1280
    labels = np.zeros(shape=(numSamples))
    i = 0
    for inputBatch, labelBatch in generator:
        featureBatch = model.predict(inputBatch)
        features[i*batchSize : (i + 1)*batchSize] = featureBatch
        labels[i*batchSize : (i + 1)*batchSize] = labelBatch
        i += 1
        if i*batchSize >= numSamples:
            break
    return features, labels

### 2.3 EfficientNet as feature extraction

In [8]:
efficientNetTrainFeatures, efficientNetTrainLabels = extractFeatures(efficientNetB0Base, baselineTrainDataGenerator, (5, 5, 1280))
efficientNetTestFeatures, efficientNetTestLabels = extractFeatures(efficientNetB0Base, baselineTestDataGenerator, (5, 5, 1280))
efficientNetValFeatures, efficientValTestLabels = extractFeatures(efficientNetB0Base, baselineValDataGenerator, (5, 5, 1280))

efficientNetTrainFeatures = efficientNetTrainFeatures.flatten()
efficientNetTestFeatures = efficientNetTestFeatures.flatten()
efficientNetValFeatures = efficientNetValFeatures.flatten()

flatFeatureSize = efficientNetTrainFeatures.size

In [12]:
efficientNetB0FEClassifier = keras.models.Sequential()
efficientNetB0FEClassifier.add(keras.layers.Dense(256, activation='relu', input_dim=flatFeatureSize))
efficientNetB0FEClassifier.add(keras.layers.Dropout(0.5))
efficientNetB0FEClassifier.add(keras.layers.Dense(1, activation='sigmoid'))

efficientNetB0FEClassifier.compile(optimizer=tfa.optimizers.Yogi,
                                  loss='binary_crossentropy',
                                  metrics=['binary_accuracy', 'binary_crossentropy', 'auc', 'precision', 'recall', 'f1'])
efficientNetB0FEClassifierHistory = efficientNetB0FEClassifier.fit(efficientNetTrainFeatures, efficientNetTrainLabels,
                                                                   epochs=30,
                                                                   batch_size=75,
                                                                   validation_data=(efficientNetTrainValFeatures, efficientNetTrainValLabels))

NameError: name 'flatFeatureSize' is not defined

In [17]:
crossEntropy, valCrossEntropy, auc, valAuc, precision, valPrecision, recall, valRecall, f1, valF1 = \
efficientNetB0FEClassifierHistory.history[('binary_crossentropy', 'val_binary_crossentropy', 'auc', 'val_auc', 'precision', 'val_precision', 'recall',
                                   'val_recall', 'f1', 'val_f1')]
epochs = range(1, len(crossEntropy) + 1)

plt.plot(epochs, crossEntropy, 'bo', label='Training Cross-Entropy')
plt.plot(epochs, valCrossEntropy, 'b', label='Validation Cross-Entropy')
plt.title('Training and validation cross-entropy for EfficientNet feature extraction')
plt.legend()
plt.savefig('EfficientNetFEXEntropy.png')

plt.figure()
plt.plot(epochs, auc, 'bo', label='Training AUC')
plt.plot(epochs, valAuc, 'b', label='Validation AUC')
plt.title('Training and validation AUC for EfficientNet feature extraction')
plt.legend()
plt.savefig('EfficientNetFEAUC.png')

plt.figure()
plt.plot(epochs, precision, 'bo', label='Training Precision')
plt.plot(epochs, valPrecision, 'b', label='Validation Precision')
plt.title('Training and validation precision for EfficientNet feature extraction')
plt.legend()
plt.savefig('EfficientNetFEPrecision.png')

plt.figure()
plt.plot(epochs, recall, 'bo', label='Training Recall')
plt.plot(epochs, valRecall, 'b', label='Validation Recall')
plt.title('Training and validation recall for EfficientNet feature extraction')
plt.legend()
plt.savefig('EfficientNetFERecall.png')

plt.figure()
plt.plot(epochs, f1, 'bo', label='Training F1-score')
plt.plot(epochs, valF1, 'b', label='Validation F1-score')
plt.title('Training and validation F1-score for EfficientNet feature extraction')
plt.legend()
plt.savefig('EfficientNetFEF1.png')

plt.show()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
